# Implementación de un pipeline RAG

Antes que cualquier otra cosa, debemos saber que un **LLM** es un tipo (modelo) de inteligencia artificial capaz de entender y generar texto en lenguaje humano. Funciona a través del aprendizaje automático y utiliza enormes cantidades de datos para aprender patrones, relaciones y contexto en el lenguaje. Esto le permite realizar tareas como traducir idiomas, responder preguntas, generar texto creativo y mucho más. En términos más sencillos, un LLM es como un asistente inteligente que puede "comprender" lo que le dices y responder de manera coherente y relevante.

## Antes de comenzar
- Debes instalar LM Studio en tu equipo. Para ello puedes seguir las indicaciones en su sitio web https://lmstudio.ai/
- Una vez instalado, debes descargar tu primer LLM. Para asegurar el funcionamiento de esta implementación, se debes descargar el modelo `google/gemma-3-12b`.
  - Para descargar un modelo debes hacer click en el ícono de Settings que aparece en la esquina inferior derecha de la ventana, y buscar el modelo mencionado en la pestaña "Model Search".
- Luego, en la esquina inferior izquierda, debes activar el modo "Power User". Con esto, aparecerá una barra lateral en el lado izquierdo de la pantalla; desde ella debes navegar a la vista "Developer".
- En la parte superior de la ventana hay un selector de modelos; en este debes seleccionar el modelo que descargamos anteriormente. Una vez que se cargue, podemos activar el servidor que disponibiliza este modelo de forma local desde la esquina superior izquierda de la vista "Developer".
- Con esto, el servidor local que disponibiliza el modelo está activo. Se puede corroborar con la información que se despliega al lado derecho de la vista "Developer".

## Comencemos a enviar prompts

Ahora podemos comenzar a interactuar con el LLM. Para esto, usando la librería LlamaIndex, vamos a conectarnos al servidor que levantamos desde LM Studio. Vamos a usar la integración de LlamaIndex para trabajar con LM Studio, especificando el modelo a ocupar y la URL que apunta al servidor.

In [ ]:
from llama_index.core import Settings
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.llms.lmstudio import LMStudio

Settings.llm = LMStudio(
    model_name="google/gemma-3-12b",
    base_url="http://localhost:1234/v1"  # This is the default LM Studio server address
)

base_conditions = [
    "Do not invent or generate any fictional data to respond to this question.", 
    "Keep your answer brief."
]

chat_engine = SimpleChatEngine.from_defaults(
    system_prompt = " ".join(base_conditions)
)

`chat_engine` nos permitirá enviar prompts al LLM y recibir una respuesta. Podemos hacer preguntas simples y el modelo las responderá de la mejor forma posible.
Incluimos un par de instrucciones preliminares para asegurar que va a responder solo con lo que sabe y no inventará información con el fin de entregar una respuesta (supuestamente) de mejor calidad, además de solicitarle que mantenga sus respuestas breves y al punto.

In [ ]:
question_samples = [
    "What is the capital of France?",
    "When did the Berlin Wall come down?",
    "What do critics say about the 2004 movie 'Mean Girls'?",
    "What do critics say about the 2025 movie 'The Fantastic Four: First Steps'?"
]
for question in question_samples:
    print(f'Question: {question}\nAnswer: {chat_engine.chat(question).response}\n--------')

Estas últimas dos preguntas denotan como, si bien los LLMs son entrenados con un volumen gigante de información, no son omniscientes. Siempre hay espacio para mejorar y entregar información nueva o ajena al conocimiento público (como podría ser lógica de un negocio, información personal, etc.)

Acá vemos como el modelo que estamos ocupando tiene noción de lo que la crítica de cine tiene que decir respecto a una película estrenada en 2004, pero no así una película estrenada hace menos de un mes (al menos desde la elaboración de este ejemplo).

Con esto introducimos la duda que nos convoca aquí hoy: ¿cómo hago que mi asistente virtual sepa cosas nuevas?

## Fine-tuning vs RAG

Digamos que tengo a mi disposición una gran biblioteca de información y quiero que mi LLM la conozca, tenga acceso a ella y haga uso de estos conocimientos al momento de responder mis preguntas. Para esto existen dos posibles caminos:

### 1. Fine-tuning al LLM

Este es el método más tradicional y potente. Implica tomar un LLM existente preentrenado, y continuar su entrenamiento en tu conjunto de datos específico.

#### Proceso
- **Preparación de datos**: Primero necesitarías extraer la inforación y formatearla en un conjunto de datos estructurado. Esto a menudo implica limpiar el texto, eliminar partes irrelevantes (como encabezados/pies de página en PDFs o elementos de navegación de sitios web), y organizarlo en un formato que el modelo pueda entender (por ejemplo, pares de preguntas y respuestas, o simplemente documentos de texto sin formato).
- **Entrenamiento del modelo**: Luego usarías un framework de machine learning (como TensorFlow, PyTorch o bibliotecas especializadas como las de Hugging Face, "transformers") para cargar tu LLM pre-entrenado y entrenarlo con tu nuevo conjunto de datos. Los pesos del modelo se actualizan en función de la nueva información, lo que efectivamente "reescribe" partes de su base de conocimiento con tus datos específicos.
- **Hardware**: Este proceso es computacionalmente intensivo y requiere importantes recursos de GPU. La cantidad de hardware necesaria depende del tamaño de tu modelo y conjunto de datos.

#### Pros
- **Integración Profunda**: El conocimiento central y las habilidades de razonamiento del modelo se infunden directamente con tu nueva información. Puede aprender matices, terminología específica y relaciones entre conceptos de tus documentos.
- **Alto Rendimiento**: Los modelos ajustados finamente pueden funcionar muy bien en tareas relacionadas con tu tema específico, a menudo generando respuestas más precisas y contextualmente apropiadas que otros métodos.
- **Capacidad sin Conexión**: Una vez ajustado finamente, el modelo no necesita acceder a los documentos originales para responder preguntas. El conocimiento está "incorporado".

#### Cons
- **Costo y Complejidad**: Este enfoque es intensivo en recursos, lo que requiere mucho tiempo, experiencia y potencia computacional.
- **Riesgo de Olvido Catastrófico**: Si no se hace con cuidado, el ajuste fino puede hacer que el modelo "olvide" parte de su conocimiento general. Es un equilibrio entre aprender el nuevo tema y retener sus habilidades existentes.


### 2. Retrieval-Augmented Generation (RAG)

Este enfoque se está volviendo cada vez más popular porque es menos costoso computacionalmente y más flexible que el fine-tuning. Implica usar el LLM como un motor de razonamiento, pero haciendo que busque la información relevante en tus documentos antes de generar una respuesta.

#### Proceso

- **Creación de Base de Datos Vectorial**: Primero procesarías toda la información que quieres introducir a la base de conocimiento y la convertirías en "incrustaciones" (embeddings). Estas son representaciones numéricas del texto que capturan su significado. Almacenas estas incrustaciones en una base de datos especializada llamada base de datos vectorial (por ejemplo, Pinecone, ChromaDB, Weaviate).
- **El Flujo de Consulta**: Cuando un usuario hace una pregunta, tu sistema realiza los siguientes pasos:
  1. La pregunta del usuario también se convierte en una incrustación vectorial.
  2. Esta incrustación se utiliza para buscar en la base de datos vectorial los fragmentos de texto más semánticamente similares de tus documentos originales.
  3. Estos fragmentos de texto relevantes se combinan con la pregunta del usuario y un prompt (por ejemplo, "Usa el siguiente contexto para responder la pregunta:") y se envían al LLM.
  4. El LLM usa este contexto proporcionado para generar una respuesta detallada y precisa.


#### Pros

- **Rentable y Escalable**: Este método no requiere reentrenar todo el modelo, lo que lo hace mucho más barato y rápido de implementar. Puedes agregar o eliminar fácilmente documentos de tu base de conocimiento simplemente actualizando la base de datos vectorial.
- **Basa al LLM**: RAG asegura que las respuestas del LLM se basen directamente en tu material fuente, reduciendo el riesgo de alucinación (inventar hechos). Es fácil proporcionar citas o fuentes para la información que utiliza.
- **Información Actualizada**: Puedes mantener fácilmente tu base de conocimiento actualizada agregando nuevos documentos a la base de datos vectorial. El fine-tuning requeriría un ciclo completo de reentrenamiento.

#### Cons

- **Razonamiento Limitado**: El LLM solo puede responder preguntas basadas en los fragmentos específicos de texto que recupera. Si la información relevante está dispersa en varios documentos o requiere una síntesis compleja, RAG podría tener dificultades en comparación con un modelo ajustado finamente que ha aprendido las relaciones entre todos los documentos.
- **Latencia**: Hay un ligero retraso ya que el sistema realiza el paso de recuperación antes de generar la respuesta final.

Para la mayoría de los casos de uso, particularmente para una primera pasada, **el enfoque RAG es altamente recomendado**. Es una forma muy práctica y potente de dar acceso a un LLM a una vasta cantidad de conocimiento específico sin la inversión significativa requerida para el fine-tuning. Muchas soluciones a nivel empresarial y startups se construyen sobre este mismo principio.

## Empecemos a usar RAG

### Fase 1: Preparación y adaptación de los datos

En este ejemplo, queremos poder preguntarle al LLM sobre la recepción crítica de la película "The Fantastic Four: First Steps". Para esto, he recopilado un conjunto de reseñas reales de esta película, los cuales se encuentran alojados en el directorio "documents" de este proyecto. Esta información ya se encuentra en texto plano, por lo que no requiere mayor preparación o limpieza preliminar.

Luego, procedemos a **separar los datos en chunks**. Este es un paso crucial. Los documentos extensos deben dividirse en fragmentos de texto más pequeños y manejables. Esto facilita la recuperación, ya que es más probable encontrar un fragmento muy relevante que intentar encontrar una aguja en un pajar. Un tamaño de fragmento común es de unos pocos cientos de palabras. Es importante que haya cierta superposición entre los fragmentos para mantener el contexto.

Por último, una vez que tengamos los fragmentos de texto, pasamos a **crear los embeddings** usando un modelo de incrustación para convertir cada fragmento en un vector numérico. Estos vectores capturan el significado semántico del texto. Existen muchas opciones, desde modelos open source hasta API comerciales de proveedores como OpenAI o Cohere.

#### Chunks?

Vamos a quedarnos acá por un momento para entender este tema de la separación de los datos en chunks. La elección del método que usaremos para llevar a cabo el chunking puede tener un impacto significativo en el performance de nuestro sistema RAG.

Las herramientas más populares y potentes para esta tarea son los módulos de *text-splitting* de LongChain y LlamaIndes. Ambos entregan una variedad de estrategias para manejar diferentes tipos de documentos y casos de uso.

Algunas de las estrategias de chuking más comunes y las librerías que se pueden usar para implementarlas son:

1. **Recursive Character Text Splitting (The Best General-Purpose Method)**

Este es el método más común y usualmente recomendado. En lugar de solo dividir el texto en chunks de un largo predeterminado, intenta preservar la lógica estructural del documento. Emplea una lista de separadores, comenzando por los más significativos (e.g., `["\n\n", "\n", " ", ""]`) e intenta separar por parrafos, luego oraciones, seguido de palabras y, por último, caracteres si el chunk es demasiado grande. Con esto procura mantener texto semanticamente relacionado junto, evitando separar oraciones o parrafos. Para este proposito, LangChain disponibiliza el módulo `RecursiveCharacterTextSplitter`, y LlamaIndex cuenta con `SentenceSplitter`.

2. **Fixed-Size Chunking (The Simplest Method)**

Este es el método más directo, donde solo se divide el texto en chunks de un largo (`chunk_size`) predeterminado, comúnmente con un traslapado (`chunk_overlap`) entre ellos. Su fortaleza recae en su facilidad de implemetar y su buen rendimiento al trabar con texto muy poco estructurado, pero su caótica división de información puede resultar en perdidas importantes de contexto. Para implementar este método, LangChain disponibiliza el módulo `CharacterTextSplitter` o `TokenTextSplitter`, y LlamaIndex cuenta con `TokenTextSplitter`.

3. **Semantic Chunking (The Most Advanced Method)**

Este método va más allá de la simple estructura del texto y utiliza el significado de este para determinar los límites de cada chunk. Comienza por separar el texto en oraciones y generar un embedding para cada una para comparar la similitud de oraciones adyacentes. Cuando detecta una caída significativa de similitud (es decir, un cambio de tema), marca el límite del chunk. Esto resulta en chunks altamente coherentes enfocados en un solo tema, lo que tiene el potencial de llevar a mejores resultados al rescatar información útil. El trade-off es que esta metodología es computacionalmente intensa al requerir la generación de embeddings para cada oración antes de llevar a cabo el proceso de chunking. Para implementar este método LlamaIndex cuenta con el módulo `SemanticSplitterNodeParser`, mientras que LangChain no disponibiliza una herramienta para llevar esta tarea, pero se puede lograr de forma custom utilizando otras herramientas de la misma.

4. **Specialized Chunking**

Para algunos tipos de data es más eficiente usar un método de chunking que entienda la estructura especifica. Por ejemplo, Markdown, HTML o código. Ambas librerías mencionadas proporcionan métodos para este tipo de chunking especializado.

Vamos a usar la primera opción, Recursive Character Text Splitting. Comenzaremos por cargar nuestros datos. Estos se encuentran en la carpeta `documents` de este proyecto.

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="documents").load_data()

Luego, intanciamos nuestro `SentenceSplitter`, indicando que queremos chunks de 250 caracteres de largo, con un traslape de 50. Obtenemos nuestros chunks invocando el método `get_nodes_from_documents` desde nuestro splitter (LlamaIndex se refiere a los chunks como *nodos*).

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(
    chunk_size=250,
    chunk_overlap=50,
)

chunks = text_splitter.get_nodes_from_documents(documents)

In [ ]:
# Puedes ejecutar el siguiente código si quieres ver los resultados del chunking
print("--- Chunking Results ---")
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk.get_content())
    print("\n" + "="*20 + "\n")

#### Embeddings?

La idea de "transformar texto en vectores" puede parecer un poco como magia, pero el entender los factores que hacen a un modelo mejor por sobre otro nos da mayor control sobre el performance de nuestro sistema.

Alguno de los aspectos más importantes a considerar cuando se busca elegir un modelo de embedding son:

1. **Performance y Benchmarks**: la forma más objetiva de comparar modelos es revisar los benchmarks. El estandar de oro en este campo es el **Massive Text Embedding Benchmark (MTEB)**, un ranking de comprensión que evalua modelos con 58 datasets en 8 tareas diferentes, incluyendo *retrieval*, *semantic textual similarity*, *clustering*, entre otras. En el caso de sistemas RAG, la métrica más importante es ***Retriveval Performance***; esta nos dice cuan bueno es un modelo para encontrar los documentos más relevantes en un corpus y ranquearlos correctamente. En corto, una calificación alta de esta medición proporciona una calidad superior al contexto que le vayamos a entregar al LLM.

2. **Tamaño del modelo y velocidad**: modelos grandes (es decir, aquellos con billones de parametros) suelen poder capturar relaciones semánticas más profundas y complejas, lo que conlleva a mayor *retrieval accuracy*. Otro factor es el tamaño de los vectores; a mayor dimensión más información se puede capturar de cada chunk, significando también necesidad de más espacio y recursos. La velocidad con la que puede transformar textos a vectores también es un factor importante a considerar. Hay que buscar un balance entre estas dos caracteristicas dependiendo de lo que se busque hacer con el modelo.

3. **Lenguaje y especificidad del dominio**: la gran mayoría de los modelos son principalmente entrenados en un corpus gigantesco de texto en Ingles. Si requieres que el sistema RAG pueda comprender textos en otros idiomas (y también varios idiomas a la vez), debes procurar utilizar un modelo de embedding capaz de entender dichos idiomas. De la misma forma, si los documentos a utilizar corresponden a información de un campo muy especializado (medicina, leyes, ciberseguridad, etc), puede que un modelo de embedding de uso general no sea capaz de entender ciertos conceptos muy especificos del área o terminología demasiado especializada. Para estos casos, procura buscar un modelo que haya sido entrenado con datos de ese dominio especifico (por ejemplo, `BioBERT` es un modelo que entiendo mucho de biomedicina).

4. **Bi-Encoders vs. Cross-Encoders**: los *bi-encoders* funcionan embebiendo la query y los documentos por separado, para luego comparar la similitud de los vectores resultantes (esto es extremadamente rápido y escalable). Por otro lado, los *cross-encoders* son un tipo diferente de modelo que embebe la query y los documentos simultaneamente, lo que le permite enteder la relación directa entre las dos partes y alcanzar resultados más precisos, a costas de significarle operar de forma lenta y poco escalable. Es por esto que los sistemas RAG en ambientes productivos suelen emplear un proceso de dos etapar llamado **re-ranking**: primero utilizan un *bi-encoder* para rescatar un alto número de los chunks más significativos, para luego usar un *cross-encoder* para volver a ranquear los chunks rescatados e identificar los realmente mejores para ser enviados al LLM.

Como nuestro ejemplo no es realmente una tarea compleja, procedemos a instanciar un modelo de embedding open-source. Este corresponde a un modelo local de Hugging Face, el cual seteamos como el que vamos a utilizar por defecto de forma global. El modelo se descargará automáticamente la primera vez que ejecutes este código.

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Vamos a crear los embeddings de forma manual para ilustrar lo que debe pasar:

In [ ]:
print("--- Creating Embeddings ---")
for i, chunk in enumerate(chunks):
    # LlamaIndex handles the embedding logic automatically.
    # We call the `get_embedding()` method on each chunk.
    embedding = Settings.embed_model.get_text_embedding(chunk.get_content())
    chunk.embedding = embedding

    print(f"Embedding created for Chunk {i+1} with shape: {len(chunk.embedding)}")

print("\n--- All chunks are now embedded! ---")

# You can now see that each node object has an `embedding` attribute
# which is a list of floating-point numbers.
# For example, let's print the first 10 numbers of the first chunk's embedding.
if chunks:
    print("\nFirst 10 values of the first chunk's embedding:")
    print(chunks[0].embedding[:10])

Realmente no necesitamos generar los embeddings manualmente, LlamaIndex de se encarga de hacer esto al generar el `index`, como veremos a continuación.

Ahora corresponde guardar los chunks embebidos en un *vector store index*. Este corresponde a una estructura de datos que permite busquedas de similitud (`similarity searches`) rápidas y eficientes. LlamaIndex cuanta con un *vector store* simple que trabaja en memoria para este propocito. En proyectos de gran escala, hay que emplear herramientas más "permanentes" como una base de datos vectorial.

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes=chunks)

Antes de continuar, debemos tener en cuenta que cada LLM funciona de manera diferente, y dentro de estas diferencias se encuentra el formato en el cual se espera que los prompts que recibe esten estructurados.

En este ejemplo estamos usando Gemma, un LLM open-source desarrollado por Google, cuyos prompts se espera que esten estructurados de una manera especifica, la cual se explica en la documentación de esta misma: https://ai.google.dev/gemma/docs/core/prompt-structure

Para poder entregar prompts que sigan esta estructura, LlamaIndex proporciona el módulo `PromptTemplate`, donde especificaremos la estructura a utilizar.

In [ ]:
from llama_index.core.prompts import PromptTemplate

custom_prompt_template = PromptTemplate(
    """<start_of_turn>user
    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Do not invent or generate any fictional data to respond to this question. 
    Keep your answer brief.
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    <end_of_turn>
    <start_of_turn>model"""
)

Por último, creamos el `query_engine` que nos permitirá enviar mensajes y recibir respuestas de nuestro LLM, pasando por nuestra base de conocimiento vectorial en busca de información extra que pueda complementar la calidad de la respuesta que el LLM nos vaya a entregar.

In [ ]:
query_engine = index.as_query_engine(
    response_mode="compact",
    text_qa_template=custom_prompt_template,
)

Volvamos a preguntar por la crítica recibida por la película "The Fantastic Four: First Steps".

In [ ]:
query = "What do critics say about the 2025 movie 'The Fantastic Four: First Steps'?"
response = query_engine.query(query)

print(f"Question: {query}")
print(f"Answer: {response}")

**BUM!**